# Blackbox Attack Notebook
------------------
This notebook contains the code used to perform black box attacks on the model to test robustness. 

It functions primarily on Google Colab, but can be adapted to work on local hardware.

It runs a uses the CIFAR 10 corrupted dataset to evaluate the F1 score of a model on a corrupted image.

The code was then refined to fit in the run.py pipeline.

In [ ]:
# STEP 1: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# STEP 2: Change directory to the OptiML folder in Google Drive
%cd /content/drive/MyDrive/OptiML

In [ ]:
# STEP 3: Import required packages

# If running in Google Colab, install necessary packages
#! pip install foolbox==3.3.1

import numpy as np
import pandas as pd
import sklearn
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from torchvision import datasets, transforms
from sklearn.metrics import accuracy_score, recall_score, f1_score
from Functions.implementations import *
from Functions.visualization import *
import foolbox as fb
from tqdm import tqdm
import os

In [ ]:
# STEP 4: Define paths for CIFAR-10-C dataset and model
#if google drive :
drive_base_path = '/content/drive/MyDrive/OptiML/repo'
cifar10_c_path = os.path.join(drive_base_path, 'CIFAR-10-C')
model_path = os.path.join(drive_base_path, 'OptML-project/Results/SGD/ResNet_Transform_lr_0.05_momentum_0.9.pth')

In [ ]:
# STEP 5: Load your model
# Replace with your actual model class
from Functions.implementations import *

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = get_resnet18_cifar().to(device)
model.load_state_dict(torch.load(model_path, map_location=device))
model.eval()

In [ ]:
# STEP 6: Load CIFAR-10-C dataset

transform = transforms.Compose([
    transforms.ToTensor(),  # only ToTensor, no normalization!
])

test_dataset = CIFAR10(root="./data", train=False, download=True, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=True)

In [ ]:
# Step 7: Wrap the model in Foolbox
mean = np.array([0.4914, 0.4822, 0.4465], dtype=np.float32)
std = np.array([0.2023, 0.1994, 0.2010], dtype=np.float32)

fmodel = fb.PyTorchModel(model, bounds=(0, 1))

# Step 5: Load one image from CIFAR-10
transform = transforms.Compose([
    transforms.ToTensor()
])

dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
image, label = dataset[0]
image = image.unsqueeze(0)  # Add batch dimension
label = torch.tensor([label])

print(image.shape)
# Step 6: Run black-box attack
attack = BoundaryAttack()

# Important: HopSkipJumpAttack works in decision-based mode and needs labels
advs, _, success = attack(fmodel, image, label, epsilons = None)

print("Attack success:", success.item())